In [1]:
#import
import numpy as np
import pandas as pd

from core import *

### Data

In [2]:
# stage_data
stage_data = pd.DataFrame({
    'stage_name': ['dough_dispenser', 'raising', 'to_oven', 'oven', 'cool_freeze'],
    'max_change_over': [15, 1, 14, 25, 2],
}, index=pd.Index([1,2,3,4,5], name='stage_id'))

stage_data

In [3]:
# recipe data
recipe_data = pd.DataFrame({
    'recipe_name': ['Baguette tradition', 'Demi Baguette', 'Ciabatta 90g','Ciabatta 300g', 'Pumpkin Seed Bun', 'Easter Bread','Brioche Bun'],
    'line_capacity': [400, 400, 400, 400, 400, 400, 400], # plb/hour
    'turner': [False, False, True, False, True, False, False],
    'rising_time': [90, 90, 90, 90, 90, 60, 60], # minute
    'baking_time': [18, 14, 12, 15, 13, 30, 12], # minute
    'baking_temp': [240, 240, 220, 220, 230, 220, 235],
    'cooling_time': [70, 70, 70, 70, 70, 70, 70] # minute
}, index=pd.Index([1, 2, 3, 4, 5, 6, 7], name='recipe_id'),)

recipe_data

In [4]:
# Processing data
processing_time_data = pd.read_excel('The Industrial Bakery Scheduling Problem.xlsx', sheet_name="processing_time", index_col=[0, 1], skiprows=50, usecols=list(range(0,3)))

processing_time_data.head(5)

In [5]:
processing_time_data["processing_time"].loc[1, 3]

In [6]:
# Change over data
change_over_data = pd.read_excel('The Industrial Bakery Scheduling Problem.xlsx', sheet_name="change_over", index_col=[0, 1], skiprows=50, usecols=list(range(0,9)))

change_over_data.head(10)

In [7]:
# production data
production_data = pd.DataFrame({
    'amount': [100000, 150000, 1250000, 70000, 50000, 25000, 45000],
    'amount_per_plb': [20, 44, 88, 27, 63, 11, 40],
    'plb_per_week': [5000, 3410, 1421, 2593, 794, 2273, 1125],
    'plb_per_day': [834, 569, 237, 433, 133, 379, 188],
}, index=pd.Index([1, 2, 3, 4, 5, 6, 7], name='recipe_id'),)

production_data

In [8]:
# Production order
production_quantity = production_data["plb_per_day"]


production_quantity = production_quantity.sample(n=7, random_state=205)

production_quantity

In [9]:
production_sequence = production_quantity.index

In [10]:
schedule = set_up_schedule(stage_data, production_sequence)

schedule

In [11]:
schedule, postpone_time = calculate_first(stage_data, recipe_data, processing_time_data, change_over_data, production_sequence, schedule)

In [12]:
schedule

##### calculating true timing

In [13]:
calculate_true(stage_data, recipe_data, processing_time_data, change_over_data, production_quantity, production_sequence, schedule, postpone_time)

##### result

In [14]:
schedule

### Makespan

In [15]:
make_schedule(stage_data, recipe_data, processing_time_data, change_over_data, production_quantity, production_sequence)

In [16]:
makespan(schedule)

## Flowshop schedule

In [17]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.operators.mutation.inversion import InversionMutation
from pymoo.termination.default import DefaultSingleObjectiveTermination
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.sampling import Sampling


#### Function for pymoo

In [18]:
class BasicScheduling(ElementwiseProblem):

    from core import make_schedule

    def __init__(self, stage_data, recipe_data, processing_time_data, change_over_data, production_quantity, **kwargs):
        """
        Basic Flowshop scheduling problem.
        """
        self.stage_data = stage_data
        self.recipe_data = recipe_data
        self.processing_time_data = processing_time_data
        self.change_over_data = change_over_data
        self.production_quantity = production_quantity
        self.make_schedule = make_schedule

        super(BasicScheduling, self).__init__(
            n_var=len(self.production_quantity),
            n_obj=1,
            xl=0,
            vtype=int,
            **kwargs
        )

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = self.schedule(x)["last_exit"].iloc[-1]

    def schedule(self, x):
        schedule = self.make_schedule(self.stage_data, self.recipe_data, self.processing_time_data, self.change_over_data, self.production_quantity, x)
        return schedule

In [19]:
class SchanttCustomSampling(Sampling):

    def _do(self, problem, n_samples, **kwargs):
        X = np.full((n_samples, problem.n_var), 0, dtype=int)
        for i in range(n_samples):
            X[i, :] = np.random.permutation(problem.production_quantity.index)
        return X

In [20]:
def create_flowshop_problem(stage_data, recipe_data, processing_time_data, change_over_data, production_quantity):
    return BasicScheduling(stage_data, recipe_data, processing_time_data, change_over_data, production_quantity)

#### Set up pymoo

In [21]:
problem = create_flowshop_problem(stage_data, recipe_data, processing_time_data, change_over_data, production_quantity)

In [22]:
termination = DefaultSingleObjectiveTermination(period=50, n_max_gen=10000)

In [23]:
algorithm = GA(
    pop_size=20,
    eliminate_duplicates=True,
    sampling=SchanttCustomSampling(),
    mutation=InversionMutation(),
    crossover=OrderCrossover()
)

In [24]:
res = minimize(
    problem,
    algorithm,
    termination,
    seed=1,
    verbose=True,
)

In [25]:
res.X

In [26]:
res.F

In [27]:
res.pop.get("X")

In [28]:
res.pop.get("F")

In [29]:
res

In [30]:
res.opt.get('X')

In [31]:
print("Maximum Span:", np.round(res.F[0], 3))
print("Function Evaluations:", res.algorithm.evaluator.n_eval)

In [32]:
best_production_sequence = res.X

In [33]:
make_schedule(stage_data, recipe_data, processing_time_data, change_over_data, production_quantity, best_production_sequence)